In [0]:
#Silver
# 2) SILVER – clean, cast, dedupe, flatten & conform

from pyspark.sql.functions import to_timestamp, col
from delta.tables           import DeltaTable

# TODO 1: point at your bronze & silver paths
bronze_path = "/tmp/bronze/doc_events/"
silver_path = "/tmp/silver/doc_events/"

# TODO 2: rm / recreate silver_path if you want truly idempotent runs
# dbutils.fs.rm(silver_path, recurse=True)

# TODO 3: read the Bronze Delta into a DataFrame
df = spark.read.format("delta").load(bronze_path)

# TODO 4: cast your string → timestamp; extract any nested fields
silver = (
  df.filter("event_time IS NOT NULL")
    .withColumn("event_ts", to_timestamp("event_time"))
    .withColumn("os",     col("device.os"))
    .withColumn("region", col("device.region"))
    .select(
       "event_id","user_id","doc_id","action",
       "event_ts","os","region","ingest_ts","batch_id"
    )
)

# TODO 5: drop duplicates on your natural key(s)
silver = silver.dropDuplicates(["event_id"])

# TODO 6: choose upsert vs overwrite vs append
if DeltaTable.isDeltaTable(spark, silver_path):
    DeltaTable.forPath(spark, silver_path) \
      .merge(
        silver.alias("new"),
        "new.event_id = event_id"
      ) \
      .whenNotMatchedInsertAll() \
      .execute()
else:
    silver.write.format("delta") \
          .mode("overwrite") \
          .save(silver_path)

# sanity-check
display(spark.read.format("delta").load(silver_path))